In [1]:
pip install convokit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.2/183.2 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.1/669.1 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 7.9 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-3.0.0-py3-none-any.whl size=216707 sha256=85c8d3187b3b2ed36b1030880014181b1af410cc25b81d1426d767cb89a98a2f
  Stored in directory: /root/.cache/pip/wheels/c4/89/8c/2677fdb888588b6f93cb6ac86bdfb020f1f1

In [2]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
pip install xgboost

In [4]:
pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 42.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [24]:
from convokit import Corpus, download
from sentence_transformers import SentenceTransformer
from ast import literal_eval
from xgboost import XGBClassifier
import numpy as np
import sklearn
from sklearn.metrics import accuracy_score, classification_report

In [6]:
corpus = Corpus(filename=download("winning-args-corpus"))

No configuration file found at /root/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
default_backend: mem


In [7]:
ids = corpus.get_utterance_ids()

print(len(ids))

293297


In [8]:
SPEAKER_BLACKLIST = ['DeltaBot','AutoModerator']
training_trios = []

for id in ids:
  ut = corpus.get_utterance(id)
  if ut.reply_to == ut.conversation_id and (ut.meta['success'] == 1 or ut.meta['success'] == 0) and (ut.speaker.id not in SPEAKER_BLACKLIST):
    op = corpus.get_utterance(ut.conversation_id).text
    x = ut.text
    y = ut.meta['success']
    training_trios += [(op, x, y)]

print(len(training_trios))

train_len = len(training_trios)

8106


In [9]:
# sentence_transformer_model = 'Alibaba-NLP/gte-large-en-v1.5'
# embedder = SentenceTransformer(sentence_transformer_model, device = "cpu", trust_remote_code=True, truncate_dim = 1024)

In [10]:
# data = []
# counter = 0


# for trio in training_trios:
#   op, x, y =  trio
#   op_vec = embedder.encode(op)
#   x_vec = embedder.encode(x)
#   data += [(op_vec, x_vec, y)]
#   counter += 1

#   if counter % 250 == 0:
#     print(f"Completed {counter}/{train_len}")

# print(f"Total training pairs is {len(data)} - expected is 8106")

In [11]:
# with open("data.txt", "w") as f: # in case colab decides to end my runtime :(
#   for trio in data:
#     op, x, y = trio

#     op_s = ", ".join(str(es) for es in op)
#     x_s = ", ".join(str(ex) for ex in x)

#     f.write(f"{op_s}::{x_s}::{y}" + "\n")

In [33]:
with open("data.txt", "r") as f:
  lines = f.readlines()

print(len(lines))

input = []
output = []
for i, line in enumerate(lines):
  parts = line.split("::")
  op, x, y = parts

  op_i = literal_eval("[" + op + "]")
  x_i = literal_eval("[" + x + "]")
  y_i = int(y)

  input += [np.array(op_i + x_i)]
  output += [y_i]


8106


In [44]:
X = np.vstack(input)
Y = np.vstack(output)

split = int(train_len*(0.7))
val = int(train_len*(0.2))

# I split it this way because I want to test on unseen arguments
train_X = X[:split]
train_Y = Y[:split]

val_X = X[split:val]
val_Y = Y[split:val]

test_X = X[val:]
test_Y = Y[val:]

randomize = np.arange(len(train_X))
np.random.shuffle(randomize)
train_X = train_X[randomize]
train_Y = train_Y[randomize]

randomize = np.arange(len(val_X))
np.random.shuffle(randomize)
val_X = val_X[randomize]
val_Y = val_Y[randomize]

randomize = np.arange(len(test_X))
np.random.shuffle(randomize)
test_X = test_X[randomize]
test_Y = test_Y[randomize]


In [50]:
model = XGBClassifier(
  max_depth=3,  # Limit tree depth to reduce model complexity
  min_child_weight=5,  # Avoid shallow trees with weak splits
  gamma=0.5,  # Minimum loss reduction for a split (prevents overfitting)
  subsample=0.8,  # Randomly sample 80% of training data per tree
  colsample_bytree=0.8,  # Randomly sample 70% of features per tree
  learning_rate=0.15,  # Smaller steps to reduce overfitting
  n_estimators=10,  # Increase number of trees to compensate for lower learning rate
)

model.fit(train_X, train_Y, eval_set = [(val_X, val_Y)], early_stopping_rounds=20)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.00000
[1]	validation_0-logloss:0.00000
[2]	validation_0-logloss:0.00000
[3]	validation_0-logloss:0.00000
[4]	validation_0-logloss:0.00000
[5]	validation_0-logloss:0.00000
[6]	validation_0-logloss:0.00000
[7]	validation_0-logloss:0.00000
[8]	validation_0-logloss:0.00000
[9]	validation_0-logloss:0.00000


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.5, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=10, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [51]:
y_predict = model.predict(test_X)
y_train_predict = model.predict(train_X)

In [52]:
print("TRAIN -------------------------")
print(accuracy_score(train_Y, y_train_predict))
print(classification_report(train_Y, y_train_predict))
print("\nTEST -------------------------")
print(accuracy_score(test_Y, y_predict))
print(classification_report(test_Y, y_predict))

TRAIN -------------------------
0.5341910468805076
              precision    recall  f1-score   support

           0       0.68      0.05      0.09      2714
           1       0.53      0.98      0.69      2960

    accuracy                           0.53      5674
   macro avg       0.60      0.51      0.39      5674
weighted avg       0.60      0.53      0.40      5674


TEST -------------------------
0.5286044718581342
              precision    recall  f1-score   support

           0       0.57      0.05      0.09      3097
           1       0.53      0.97      0.68      3388

    accuracy                           0.53      6485
   macro avg       0.55      0.51      0.39      6485
weighted avg       0.55      0.53      0.40      6485

